In [1]:
import pandas as pd

In [2]:
path = "./logs/"

In [3]:
def treatMsg(linha):
    linha = linha.strip()
    dados = linha.split(", data=")
    data = [int(x) for x in dados[1][1:-2].split(",")]
    dados = dados[0].split(":sendBSMsg: Sender= ")
    [sender,evtId] = [int(x) for x in dados[1].split(", EvtId=")]
    date = dados[0]
    d = {"datetime":date, "sender":sender, "evtId":evtId, "seq":data[0], "temp":data[4], "pai":data[5], "dados":data}
    return d

#Function to read a .txt file to read the messages
def readMsg(filename):
    #print("Reading file: " + filename)
    file = open(path+filename, "r")
    lines = file.readlines()
    file.close()
    listLines = []
    for line in lines:
        d = treatMsg(line)
        listLines.append(d)
    return pd.DataFrame(listLines)

def calcula(df):
    qtdNan = df.isna().sum().sum()
    total = df.shape[0]*df.shape[1]
    return 1-qtdNan/total


def duplicados(dataframe):
    dataframe_sem_nan = dataframe.fillna(0)
    quantidade_valores_maiores_que_um = (dataframe_sem_nan > 1).sum().sum()
    return quantidade_valores_maiores_que_um

def contar_nan_ate_ultimas_linhas(dataframe):
    ultimos_indices_nao_nan = dataframe.apply(lambda coluna: coluna.last_valid_index())
    
    contagem_nan_por_coluna = {}
    for coluna in dataframe.columns:
        contagem_nan = dataframe[coluna].loc[:ultimos_indices_nao_nan[coluna]].isna().sum()
        contagem_nan_por_coluna[coluna] = contagem_nan
    
    return contagem_nan_por_coluna


def indices_ultimas_linhas_nao_nan(dataframe):
    # Encontrar os índices das últimas linhas não NaN para cada coluna
    ultimos_indices_nao_nan = dataframe.apply(lambda coluna: coluna.last_valid_index())
    
    return ultimos_indices_nao_nan

def contar_nan_ate_ultimas_linhas(dataframe):
    # Obter os índices das últimas linhas não NaN para cada coluna
    ultimos_indices_nao_nan = indices_ultimas_linhas_nao_nan(dataframe)
    
    # Inicializar um dicionário para armazenar a contagem de NaN por coluna
    contagem_nan_por_coluna = {}
    
    # Iterar sobre as colunas do DataFrame
    for coluna in dataframe.columns:
        # Obter a contagem de NaN até o último índice não NaN
        contagem_nan = dataframe[coluna].loc[:ultimos_indices_nao_nan[coluna]].isna().sum()
        
        # Armazenar a contagem no dicionário
        contagem_nan_por_coluna[coluna] = contagem_nan
    
    return contagem_nan_por_coluna

def calc_qtd_tentativas(df):
    d=indices_ultimas_linhas_nao_nan(df).to_dict()
    soma=0
    for key in d:
        soma+=d[key]
    return soma

def calc_qtd_perdidos(df):
    d=contar_nan_ate_ultimas_linhas(df)
    soma=0
    for key in d:
        soma+=d[key]
    return soma


def exibe_metricas_log(filename):
    df = readMsg(filename)
    dfpv = pd.pivot_table(df, index="seq",columns="sender",values="temp",aggfunc="count")
    qtdPerd=calc_qtd_perdidos(dfpv)
    qtdTent=calc_qtd_tentativas(dfpv)
    
    print("Quatidade de nós:",dfpv.shape[1])
    print("Quantidado de sequências:",dfpv.shape[0])
    print("Número da sequência máxima:",dfpv.iloc[-1].name)
    print("Quantidade de valores duplicados:",duplicados(dfpv))
    print("Quantidade de tentativas conhecidas de envio:",qtdTent)
    print("Quantidade de mensagens com certeza perdidas:",qtdPerd)
    print("Percentual de tentativas conhecidas recebidas: %.1f%%"%(100*(1-qtdPerd/qtdTent)))
    
    print("Percentual de mensagens recebidas:")
    print("\tTotal: %.1f%%"%(calcula(dfpv)*100))
    print("\t[-1]:  %.1f%%"%(calcula(dfpv[:-1])*100))
    print("\t[-2]:  %.1f%%"%(calcula(dfpv[:-2])*100))
    print("\t[-3]:  %.1f%%"%(calcula(dfpv[:-3])*100))

In [4]:
exibe_metricas_log("log5.txt")

Quatidade de nós: 16
Quantidado de sequências: 11
Número da sequência máxima: 10
Quantidade de valores duplicados: 0
Quantidade de tentativas conhecidas de envio: 104
Quantidade de mensagens com certeza perdidas: 6
Percentual de tentativas conhecidas recebidas: 94.2%
Percentual de mensagens recebidas:
	Total: 64.8%
	[-1]:  70.6%
	[-2]:  75.7%
	[-3]:  80.5%


In [225]:
df = readMsg("log3.txt")
# df.loc[0]["dados"][4]
df

,datetime,sender,evtId,seq,temp,pai,dados
0,2023.11.10 20:18:34,11,1,1,500,1,"[1, 0, 0, 0, 500, 1, 0, 0, 0, 0]"
1,2023.11.10 20:18:34,22,1,0,500,11,"[0, 0, 0, 0, 500, 11, 0, 0, 0, 0]"
2,2023.11.10 20:18:34,32,1,0,500,22,"[0, 0, 0, 0, 500, 22, 0, 0, 0, 0]"
3,2023.11.10 20:18:34,12,1,0,500,11,"[0, 0, 0, 0, 500, 11, 0, 0, 0, 0]"
4,2023.11.10 20:18:35,33,1,0,500,32,"[0, 0, 0, 0, 500, 32, 0, 0, 0, 0]"
...,...,...,...,...,...,...,...
600,2023.11.10 20:29:00,23,1,64,500,22,"[64, 0, 0, 0, 500, 22, 0, 0, 0, 0]"
601,2023.11.10 20:29:00,41,1,63,500,31,"[63, 0, 0, 0, 500, 31, 0, 0, 0, 0]"
602,2023.11.10 20:29:00,34,1,63,500,23,"[63, 0, 0, 0, 500, 23, 0, 0, 0, 0]"
603,2023.11.10 20:29:00,42,1,64,500,32,"[64, 0, 0, 0, 500, 32, 0, 0, 0, 0]"


In [226]:
dfpv = pd.pivot_table(df, index="seq",columns="sender",values="temp",aggfunc="count")
dfpv

sender,11,12,13,14,21,22,23,24,31,32,33,34,41,42,43,44
seq,,,,,,,,,,,,,,,,
0,NaN,1.0,1.0,NaN,NaN,1.0,NaN,NaN,NaN,1.0,1.0,1.0,NaN,1.0,NaN,1.0
1,1.0,NaN,1.0,NaN,1.0,1.0,1.0,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,NaN
2,1.0,NaN,1.0,NaN,1.0,1.0,NaN,1.0,NaN,1.0,NaN,1.0,1.0,1.0,NaN,NaN
3,1.0,NaN,1.0,NaN,1.0,1.0,NaN,NaN,NaN,1.0,1.0,1.0,NaN,1.0,1.0,NaN
4,1.0,1.0,NaN,NaN,NaN,1.0,1.0,NaN,1.0,1.0,1.0,1.0,NaN,NaN,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,1.0,NaN,1.0,1.0,NaN,1.0,NaN,1.0,NaN,1.0,NaN,NaN,NaN,1.0,1.0,1.0
62,1.0,NaN,1.0,NaN,1.0,1.0,NaN,1.0,NaN,1.0,NaN,1.0,1.0,1.0,1.0,NaN
63,1.0,NaN,1.0,NaN,NaN,1.0,NaN,1.0,NaN,1.0,NaN,1.0,1.0,1.0,NaN,1.0


In [227]:
calcula(dfpv[:-1])

0.5807692307692307

In [228]:
duplicados(dfpv)

0

In [229]:
calc_qtd_perdidos(dfpv)

369

In [230]:
calc_qtd_tentativas(dfpv)

958

In [208]:
def indices_ultimas_linhas_nao_nan(dataframe):
    # Encontrar os índices das últimas linhas não NaN para cada coluna
    ultimos_indices_nao_nan = dataframe.apply(lambda coluna: coluna.last_valid_index())
    
    return ultimos_indices_nao_nan

def contar_nan_ate_ultimas_linhas(dataframe):
    # Obter os índices das últimas linhas não NaN para cada coluna
    ultimos_indices_nao_nan = indices_ultimas_linhas_nao_nan(dataframe)
    
    # Inicializar um dicionário para armazenar a contagem de NaN por coluna
    contagem_nan_por_coluna = {}
    
    # Iterar sobre as colunas do DataFrame
    for coluna in dataframe.columns:
        # Obter a contagem de NaN até o último índice não NaN
        contagem_nan = dataframe[coluna].loc[:ultimos_indices_nao_nan[coluna]].isna().sum()
        
        # Armazenar a contagem no dicionário
        contagem_nan_por_coluna[coluna] = contagem_nan
    
    return contagem_nan_por_coluna

def calc_qtd_tentativas(df):
    d=indices_ultimas_linhas_nao_nan(df).to_dict()
    soma=0
    for key in d:
        soma+=d[key]
    return soma

def calc_qtd_perdidos(df):
    d=contar_nan_ate_ultimas_linhas(dfpv)
    soma=0
    for key in d:
        soma+=d[key]
    return soma

In [209]:
calc_qtd_perdidos(dfpv)

28

In [210]:
calc_qtd_perdidos(dfpv)
calc_qtd_tentativas(dfpv)

81

In [203]:
indices_ultimas_linhas_nao_nan(dfpv).to_dict()

{11: 7,
 12: 5,
 13: 5,
 14: 5,
 21: 5,
 22: 6,
 23: 6,
 24: 5,
 31: 4,
 32: 6,
 33: 4,
 34: 5,
 41: 2,
 42: 6,
 43: 5,
 44: 5}

In [186]:
indices_ultimas_linhas_nao_nan(dfpv)
d=contar_nan_ate_ultimas_linhas(dfpv)
d

{11: 2,
 12: 3,
 13: 1,
 14: 4,
 21: 1,
 22: 1,
 23: 2,
 24: 3,
 31: 3,
 32: 1,
 33: 1,
 34: 0,
 41: 0,
 42: 2,
 43: 1,
 44: 3}

In [187]:
soma=0
for key in d:
    soma+=d[key]
soma

28